In [4]:

import zipfile
import pandas as pd
from pathlib import Path
import urllib.request

SEED = 42


def load_kaggle_data():
    file_path = Path("./datasets/archive.zip")
    if not file_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://www.kaggle.com/api/v1/datasets/download/amirmahdiabbootalebi/salary-by-job-title-and-country"
        urllib.request.urlretrieve(url, file_path)
    zf = zipfile.ZipFile("./datasets/archive.zip")

    return pd.read_csv(zf.open('Salary.csv'))

kaggle_data = load_kaggle_data ()
print (*kaggle_data.columns, sep='\n')

Age
Gender
Education Level
Job Title
Years of Experience
Salary
Country
Race
Senior


In [5]:
from faker import Faker
import random

# Copy dataset and only take the columns we need
dataset_clean = kaggle_data[["Salary", "Country", "Job Title"]]

# Rename the columns that were copied
dataset_clean = dataset_clean.rename (columns={"Salary": "salary", "Country": "work_location", "Job Title": "job_role"})

# Create mock data for the remaininig columns
Faker.seed(SEED)
faker = Faker()

dataset_clean["name"] = [faker.name() for i in range (len (dataset_clean))]
dataset_clean["phone_number"] = [f"{random.randint(200, 999):03d}{random.randint(200, 999):03d}{random.randint(200, 999):04d}" for i in range (len (dataset_clean))]

print (*dataset_clean.columns, sep="\n")
print (dataset_clean.head ())

salary
work_location
job_role
name
phone_number
     salary work_location           job_role             name phone_number
0   90000.0            UK  Software Engineer     Allison Hill   6367040582
1   65000.0           USA       Data Analyst      Noah Rhodes   4264170782
2  150000.0        Canada            Manager  Angie Henderson   9722800202
3   60000.0           USA    Sales Associate    Daniel Wagner   4206430694
4  200000.0           USA           Director  Cristian Santos   3924710750


In [6]:
# Save the data to the disk
dataset_clean.to_parquet ("./datasets/dataset_clean.parquet")